<a href="https://colab.research.google.com/github/KseniyaShilova/rl-game/blob/main/rl_game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import random
from collections import deque

import numpy as np
import torch
from gym import make
from torch import nn
from torch.optim import Adam, lr_scheduler

In [3]:
GAMMA = 0.99
INITIAL_STEPS = 1024
TRANSITIONS = 2_000_000
STEPS_PER_UPDATE = 4
STEPS_PER_TARGET_UPDATE = STEPS_PER_UPDATE * 1000
BATCH_SIZE = 128
BUFFER_SIZE = 128_000
LEARNING_RATE = 5e-4
DEVICE = torch.device("cuda")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
!git clone https://github.com/Farama-Foundation/gym-examples
# cd gym-examples

# source .env/bin/activate
!pip install -e .

Cloning into 'gym-examples'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 98 (delta 34), reused 23 (delta 23), pack-reused 39
Receiving objects: 100% (98/98), 18.43 KiB | 6.14 MiB/s, done.
Resolving deltas: 100% (39/39), done.
Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [8]:
class QModel(nn.Module):
    def __init__(self, state_dim, action_dim, fc1=512, fc2=512):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(state_dim, fc1),
            nn.BatchNorm1d(fc1),
            nn.ReLU(),
            nn.Linear(fc1, fc2),
            nn.BatchNorm1d(fc2),
            nn.ReLU(),
            nn.Linear(fc2, action_dim),
        )

    def forward(self, x):
        return self.fc(x)

    @staticmethod
    def weights_init(m):
        classname = m.__class__.__name__
        if classname.find('Linear') != -1:
            nn.init.xavier_uniform_(m.weight)
